In [36]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
model_name="llm-jp/llm-jp-13b-instruct-full-jaster-v1.0"
model_name="kanhatakeyama/Tanuki-ZeRo"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto",
                                              torch_dtype=torch.float16)


Loading checkpoint shards: 100%|██████████| 11/11 [00:06<00:00,  1.74it/s]


In [38]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              max_new_tokens=1,
              pad_token_id=7
              )

In [39]:
instruction="日本語の文ペアの意味がどのくらい近いかを判定し、類似度を0.0〜5.0までの間の値で付与してください。0.0に近いほど文ペアの意味が異なり、5.0に近いほど文ペアの意味が似ていることを表しています。整数値のみを返し、それ以外には何も含めないことを厳守してください。"
t1=""
t2=""
problem="文1：レンガの建物の前を、乳母車を押した女性が歩いています。\n文2：厩舎で馬と女性とが寄り添っています。"
problem_template="文1：{t1}\n文2：{t2}"

In [40]:
txt="指示:" +instruction+"\n入力: "+ problem+"\n出力: "


In [41]:
import pandas as pd
df=pd.read_csv("../datasets/JSNLI_train.tsv",sep="\t",
              )
df.columns=["label","sentence1","sentence2"]
df["sentence1"]=df["sentence1"].str.replace(" ","")
df["sentence2"]=df["sentence2"].str.replace(" ","")

#shuffle dataframe
df=df.sample(frac=1)
df

,label,sentence1,sentence2
457894,neutral,２人の友人が丘を自転車で下ります。,丘の上の２人の男の子。
172567,entailment,アイスクリームトラックを渡す女性が頭に帽子をかぶっています。,女性は頭に手をかざして帽子をかぶっています。
75176,contradiction,２人の男性が、大きくて装飾的なヨットを背景にしたネットで働いています。,二人の男がヨットでネットに登る。
201161,entailment,旗を掲げた男性は、カメラを通り過ぎるときに１本の指を持っていました。,男は歩いていた。
132639,contradiction,買い物袋を持って歩道を歩いている老人。,老人は雪の丘をスキーします。
...,...,...,...
285492,contradiction,ライフガードのグループは、ビーチの海岸線に沿ってハイチェアに一緒に座っています。,ライフガードはプールで泳いでいた。
227963,entailment,２匹の犬がビーチに沿って追いかけます。,二匹の犬がビーチにいます
100523,neutral,灰色のシャツを着た酔っぱらいは、もう１本の樽を待っている間に、ビールの樽に穴を開けようとして...,彼はビールを飲むことができるように樽を掘っています。
134950,entailment,３人の乗組員が落書きをペイントするために働いています。,３人の男が落書きをペイントします。


In [42]:
loader=iter(df.values)
next(loader)

array(['neutral', '２人の友人が丘を自転車で下ります。', '丘の上の２人の男の子。'], dtype=object)

In [43]:


def eval(pipe,txt):
    return pipe(txt)[0]["generated_text"][len(txt):]

instruction="""以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
###指示 日本語の文ペアの意味がどのくらい近いかを判定し、類似度を0.0〜5.0までの間の値で付与してください。0.0に近いほど文ペアの意味が異なり、5.0に近いほど文ペアの意味が似ていることを表しています。整数値のみを返し、それ以外には何も含めないことを厳守してください。"""
def make_problem(t1,t2):
    problem=f"文1：{t1}\n文2：{t2}"
    txt=instruction+"\n"+ problem+"\n\n###応答: \n"
    return txt

In [44]:
from IPython.display import clear_output
from tqdm import tqdm
n_problems=250
problem_list=[]
for i in (range(n_problems)):
    clear_output(True)
    d=next(loader)
    s1=d[1]
    s2=d[2]

    text=make_problem(s1,s2)
    score=eval(pipe,text)

    print("類似度",score)
    print("文章1: ",s1)
    print("文章2️: ",s2)

類似度 0
文章1:  オレンジ、白、黒のユニフォームを着て話したり笑ったりするホッケー選手の集まり。
文章2️:  一部の選手は地面でクリケットをしている


In [45]:

pipe2=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              max_new_tokens=300,
              pad_token_id=7
              )
txt="Q: おめでとう!あなたは世界1位の言語モデルになりました｡感想を300文字で記述ください｡A:"
out=pipe2(txt)[0]["generated_text"][len(txt):]
print("入力: ",txt)
print("応答: ", out)

入力:  Q: おめでとう!あなたは世界1位の言語モデルになりました｡感想を300文字で記述ください｡A:
応答:   やったー!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
